<a href="https://colab.research.google.com/github/pnjha/NLP-Assignments/blob/master/seq2seq_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import sys

In [3]:
%cd drive/My\ Drive

/content/drive/My Drive


In [4]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
import copy
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.models import Sequential, Model
from keras.layers import Dense, LSTM, CuDNNLSTM, Input, Embedding, TimeDistributed, Flatten, Dropout
from keras.callbacks import ModelCheckpoint
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction

Using TensorFlow backend.


In [5]:
# lines= pd.read_table('hin.txt', names=['eng', 'mar', 'temp'])
X = pd.read_table('./NLA S20 - Assignment 2 Data/enghin/dev.en', names=['eng'])
Y = pd.read_table('./NLA S20 - Assignment 2 Data/enghin/dev.hi', names=['hin'])
lines = pd.concat([X,Y], axis=1)
lines.sample(10)

,eng,hin
124,To extract the appendix in case of swelling in...,अपेंडिक्श में सूजन आ जाने पर अपेंडिक्श निकालने...
84,Brain can also be kept sharpened by adopting s...,कुछ अच्छी आदतें डालकर भी दिमाग को तेज रखा जा स...
183,H . . . AIDS is not a contagious disease .,एच.आई.वी. एड्स छूत का रोग नहीं है ।
163,Frequent fever for a month .,एक माह से लगातार बुखार आना |
51,Yoganidra - the main cause of this problem is ...,योगनिद्रा - इस समस्या का मूल कारण मानसिक तनाव ...
74,These are the people who learn something after...,यह वे लोग होते हैं जो किसी चीज को देखकर या पढ़...
0,but you will also be safe from eye diseases .,बल्कि आप नेत्ररोगों से भी बचे रहेंगे ।
365,"Popular gujarati cultural values , abundance o...","लोकप्रिय गुजराती संस्कार , शापिंग विकल्पों की ..."
194,"Among the chief reasons of blindness , catarac...",दृष्टिहीनता के मुख्य कारणों में मोतियाबिंद 55 ...
140,Patients are administered more anesthetic medi...,"मरीज को ज्यादा निश्चेतक दवा देनी पड़ती है , जिस..."


In [6]:
print(lines.shape)
print(lines.eng.shape)
print(lines.hin.shape)

(401, 2)
(401,)
(401,)


In [0]:
def process_data(data,append_char):
  data = data.apply(lambda x: x.lower())
  data = data.apply(lambda x: x.strip())
  data = data.apply(lambda x: re.sub("'", '', x))
  exclude = set(string.punctuation) # Set of all special characters
  data = data.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
  remove_digits = str.maketrans('', '', digits)
  data = data.apply(lambda x: x.translate(remove_digits))
  if append_char:
    data = data.apply(lambda x : 'START_ '+ x + ' _END')
  return data

In [0]:
lines.eng = process_data(lines.eng,False)
lines.hin = process_data(lines.hin,True)

In [0]:
# Lowercase all characters
# lines.eng=lines.eng.apply(lambda x: x.lower())
# lines.hin=lines.hin.apply(lambda x: x.lower())
# X.eng=X.eng.apply(lambda x: x.lower())
# Y.hin=Y.hin.apply(lambda x: x.lower())

In [0]:
# Remove quotes
# lines.eng=lines.eng.apply(lambda x: re.sub("'", '', x))
# lines.hin=lines.hin.apply(lambda x: re.sub("'", '', x))
# X.eng=X.eng.apply(lambda x: re.sub("'", '', x))
# Y.hin=Y.hin.apply(lambda x: re.sub("'", '', x))

In [0]:
# exclude = set(string.punctuation) # Set of all special characters
# # Remove all the special characters
# lines.eng=lines.eng.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
# lines.hin=lines.hin.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
# lines.eng=lines.eng.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
# lines.hin=lines.hin.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [0]:
# Remove all numbers from text
# remove_digits = str.maketrans('', '', digits)
# lines.eng=lines.eng.apply(lambda x: x.translate(remove_digits))
# lines.hin = lines.hin.apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))

In [0]:
# Remove extra spaces
# lines.eng=lines.eng.apply(lambda x: x.strip())
# lines.hin=lines.hin.apply(lambda x: x.strip())
# lines.eng=lines.eng.apply(lambda x: re.sub(" +", " ", x))
# lines.hin=lines.hin.apply(lambda x: re.sub(" +", " ", x))

In [0]:
# Add start and end tokens to target sequences
# lines.hin = lines.hin.apply(lambda x : 'START_ '+ x + ' _END')

In [15]:
lines.sample(10)

,eng,hin
2,common blindness causing diseases like catarac...,START_ कि खाने में विटामिनसी और ओमेगा तेलों व ...
9,you must eat two or three eggs daily in breakf...,START_ नाश्ते में रोज दो या तीन अंडे जरूर खाएँ...
212,from this route tourists go to pasupathinath t...,START_ इसी रूट से पर्यटक पशुपतिनाथ बाजार भी जा...
349,the idols built here are unique and look so li...,START_ यहाँ निर्मित मूर्तियाँ अद्वितीय हैं और ...
372,this city is famous for its parks temples mu...,START_ यह शहर अपने पार्क मंदिरों म्यूजियमों ...
292,if you have to go to toilet frequently then ta...,START_ अगर आपको बारबार लू जाना पड़ता है तो कोन...
251,first of all a walk with ram babu and the jeep...,START_ सबसे पहले राम बाबू के साथ पैदल सैर और फ...
398,entertainment is along everytime,START_ हर वक्‍त का मनोरंजन साथ में । _END
160,hundreds of such small creatures live in our b...,START_ हमारे शरीर में सैकड़ों ऐसे छोटे जीव रहत...
319,beside the memorial not only of rome but of en...,START_ स्मारक के साथ ही सिर्फ रोम का ही नहीं ब...


In [0]:
# Vocabulary of English
all_eng_words=set()
for eng in lines.eng:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

# Vocabulary of Hindi 
all_hindi_words=set()
for hin in lines.hin:
    for word in hin.split():
        if word not in all_hindi_words:
            all_hindi_words.add(word)

In [17]:
# Max Length of source sequence
lenght_list=[]
for l in lines.eng:
    lenght_list.append(len(l.split(' ')))
max_length_src = np.max(lenght_list)
max_length_src

50

In [18]:
# Max Length of target sequence
lenght_list=[]
for l in lines.hin:
    lenght_list.append(len(l.split(' ')))
max_length_tar = np.max(lenght_list)
max_length_tar

50

In [19]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_hindi_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_hindi_words)
num_encoder_tokens, num_decoder_tokens

(1700, 1966)

In [20]:
num_decoder_tokens += 1 # For zero padding
num_decoder_tokens

1967

In [0]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [0]:

reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [23]:
lines = shuffle(lines)
lines.head(10)

,eng,hin
253,saw a dry lake in which farmers were farming,START_ सूखी झील देखी जिसमें किसान खेती कर रहे ...
47,do the same exercise another side also,START_ यही क्रिया दूसरी तरफ भी करें । _END
195,generally pure lens or its capsule becomes dim...,START_ जब साधारणतया स्वच्छ लैन्स या उसका कैपस्...
121,generally pap starts developing in girls after...,START_ आमतौर पर पैप लड़कियों में साल की उम्र क...
311,rome is famous worldwide for ancient monuments...,START_ प्राचीन इमारतों और उनके अवशेषों के लिए ...
261,i have heard tour packages are organized to s...,START_ मैंने सुना है ट्राइबल लाइफ दिखाने के ल...
237,it s a different matter that she herself reach...,START_ यह बात अलग है कि वे खुद राह भटक कर भानग...
182,why discrimination when all the patients are e...,START_ एक डाक्टर के लिए सभी मरीज एक से हैं फिर...
361,in the journey from gujarat abutting to the we...,START_ पश्‍चिमी तट से लगे हुए गुजरात से गोवा त...
188,the main reason of blindness is cataract which...,START_ दृष्टिहीनता का मुख्य कारण मोतियाबिंद है...


In [24]:
# Train - Test Split
X, y = lines.eng, lines.hin
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)
l = X_train.values.tolist()
X_train_normal = cpoy.deepcopy(X_train)
t = []
for item in l:
  p = l.split()
  reverse(p)
  p = " ".join(p)
  t.append(p)
df = pd.DataFrame({'data':t})
X_train = df.data
X_train.shape, X_test.shape

((360,), (41,))

In [0]:
X_train.to_pickle('X_train_eng_hin.pkl')
X_test.to_pickle('X_test_eng_hin.pkl')

In [0]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [0]:
latent_dim = 1000
vec_len = 200
dropout_rate = 0.2

In [28]:
# encoder_inputs = Input(shape=(None,))
# encoder_embedding = Embedding(input_dim = num_encoder_tokens, output_dim = vec_len)(encoder_inputs)
# encoder_dropout   = (TimeDistributed(Dropout(rate = dropout_rate)))(encoder_embedding)

# encoder_LSTM = CuDNNLSTM(latent_dim, return_sequences=True)(encoder_dropout)
# encoder_LSTM_2 = CuDNNLSTM(latent_dim, return_sequences=True)(encoder_LSTM)
# encoder_outputs, state_h, state_c = CuDNNLSTM(latent_dim, return_state=True)(encoder_LSTM_2)

# encoder_states = [state_h, state_c]

encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
# Set up the decoder, using `encoder_states` as initial state.
# decoder_inputs = Input(shape=(None,))
# dec_emb_layer = Embedding(num_decoder_tokens,output_dim = vec_len)
# dec_emb = dec_emb_layer(decoder_inputs)

# decoder_layer_1 = CuDNNLSTM(latent_dim, return_sequences=True)(dec_emb,initial_state=encoder_states)
# decoder_lstm = CuDNNLSTM(latent_dim, return_sequences=True, return_state=True)
# decoder_outputs, _, _ = decoder_lstm(decoder_layer_1)
# decoder_dense = Dense(num_decoder_tokens, activation='softmax')
# decoder_outputs = decoder_dense(decoder_outputs)

# # Define the model that will turn
# # `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
# model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [30]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [0]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 64
epochs = 80

In [32]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples/batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples/batch_size)




Epoch 1/80





6/5 [================================] - 12s 2s/step - loss: 7.2426 - acc: 0.0636 - val_loss: 6.6301 - val_acc: 0.0606
Epoch 2/80
6/5 [================================] - 1s 216ms/step - loss: 6.1675 - acc: 0.1018 - val_loss: 6.3821 - val_acc: 0.1094
Epoch 3/80
6/5 [================================] - 1s 214ms/step - loss: 5.9388 - acc: 0.1270 - val_loss: 6.4215 - val_acc: 0.1146
Epoch 4/80
6/5 [================================] - 1s 212ms/step - loss: 5.7711 - acc: 0.1375 - val_loss: 6.3962 - val_acc: 0.1212
Epoch 5/80
6/5 [================================] - 1s 214ms/step - loss: 5.5810 - acc: 0.1468 - val_loss: 6.4164 - val_acc: 0.1186
Epoch 6/80
6/5 [================================] - 1s 207ms/step - loss: 5.4231 - acc: 0.1602 - val_loss: 6.4104 - val_acc: 0.1436
Epoch 7/80
6/5 [================================] - 1s 214ms/step - loss: 5.3677 - acc: 0.1628 - val_loss: 6.3895 - val_acc: 0.1489
Epoch 8/80
6/5 [================================] - 1s 217ms/step - lo

In [0]:
model.save_weights('nmt_weights.h5')

In [0]:
model.load_weights('nmt_weights.h5')

In [35]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 1000)   1700000     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 1000)   1967000     input_2[0][0]                    
____________________________________________________________________________________________

In [0]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [0]:
def calculate_bleu_score(actual_string, predicted_string):
  actual_string = copy.deepcopy(actual_string)
  predicted_string = copy.deepcopy(predicted_string)
  reference = re.split("\s",actual_string.strip())
  candidate = re.split("\s",predicted_string.strip())
  smoothie = SmoothingFunction().method4
  score = sentence_bleu(reference, candidate, smoothing_function=smoothie)
  return score

In [0]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [0]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [40]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])
print("bleu score: ", calculate_bleu_score(y_train[k:k+1].values[0][6:-4],decoded_sentence[:-4]))

Input English sentence: if you keep an interest in artefacts then do look at it 
Actual Hindi Translation:  अगर आप कलाकृतियों में दिलचस्पी रखते हैं  तो उसे जरूर देखें । 
Predicted Hindi Translation:  अगर आप कलाकृतियों में दिलचस्पी रखते हैं तो उसे 
bleu score:  0


In [0]:
val_gen = generate_batch(X_test, y_test, batch_size = 1)
k=-1

In [42]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Hindi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])
print("bleu score: ", calculate_bleu_score(y_train[k:k+1].values[0][6:-4],decoded_sentence[:-4]))

Input English sentence: frequent loose motion for a month 
Actual Hindi Translation:  एक माह से लगातार दस्त आना  
Predicted Hindi Translation:  एक माह से लगातार बुखार आना 
bleu score:  0
